In [2]:
import torch 
import torch.nn.functional as F
import numpy as np
from torch import nn

In [4]:
# this is copy pasted from tutorial
from __future__ import unicode_literals, print_function, division
from io import open
import glob
import os
import unicodedata
import string

all_letters = string.ascii_letters + " .,;'-"
n_letters = len(all_letters) + 1 # Plus EOS marker

def findFiles(path): return glob.glob(path)

# Turn a Unicode string to plain ASCII, thanks to https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
        and c in all_letters
    )

# Read a file and split into lines
def readLines(filename):
    with open(filename, encoding='utf-8') as some_file:
        return [unicodeToAscii(line.strip()) for line in some_file]

# Build the category_lines dictionary, a list of lines per category
category_lines = {}
all_categories = []
for filename in findFiles('../char-rnn-classify/data/names/*.txt'):
    category = os.path.splitext(os.path.basename(filename))[0]
    all_categories.append(category)
    lines = readLines(filename)
    category_lines[category] = lines

n_categories = len(all_categories)

if n_categories == 0:
    raise RuntimeError('Data not found. Make sure that you downloaded data '
        'from https://download.pytorch.org/tutorial/data.zip and extract it to '
        'the current directory.')

print('# categories:', n_categories, all_categories)
print(unicodeToAscii("O'Néàl"))

# categories: 18 ['Czech', 'German', 'Arabic', 'Japanese', 'Chinese', 'Vietnamese', 'Russian', 'French', 'Irish', 'English', 'Spanish', 'Greek', 'Italian', 'Portuguese', 'Scottish', 'Dutch', 'Korean', 'Polish']
O'Neal


In [213]:
class Model(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, n_categories):
        super(Model,self).__init__()
        self.hidden_size = hidden_size
        
        self.i2o = nn.Linear(n_categories + input_size + hidden_size, output_size)
        self.i2h = nn.Linear(n_categories + input_size + hidden_size, hidden_size)

        self.o2o = nn.Linear(output_size + hidden_size, output_size)

        self.dropout = nn.Dropout(0.1)
        
    
    def forward(self, category, input, hidden):        
        combined = torch.cat((category, input, hidden), 1)

        out = self.i2o(combined)
        hidden = self.i2h(combined)

        combined = torch.cat((out, hidden), 1)

        out = self.o2o(combined)
        out = self.dropout(out)
        out = F.log_softmax(out, dim=1)

        return (out, hidden)
    
    def init_hidden(self):
        return torch.zeros(1, self.hidden_size)


In [198]:
# Find letter index from all_letters, e.g. "a" = 0
def letterToIndex(letter):
    return all_letters.find(letter)

# Just for demonstration, turn a letter into a <1 x n_letters> Tensor
def letterToTensor(letter):
    tensor = torch.zeros(1, n_letters)
    tensor[0][letterToIndex(letter)] = 1
    return tensor

# Turn a line into a <line_length x 1 x n_letters>,
# or an array of one-hot letter vectors
def lineToTensor(line):
    tensor = torch.zeros(len(line), 1, n_letters)
    for li, letter in enumerate(line):
        tensor[li][0][letterToIndex(letter)] = 1
    return tensor


# Prepare the dataset

1. For each word hot encode it, add 1 special EOS char in the end.
2. Split each word into first n-1 and last n-1 chars.
3. Each batch will be just one word. Feed consecutively each letter and accumulate loss over the word and then call backwards.

In [20]:
def getCategoryIndex(category, all_categories):
    return all_categories.index(category)

def encodeCategory(category, all_categories, all_categories_size):
    index = getCategoryIndex(category, all_categories)
    tensor = torch.zeros(1, all_categories_size)
    tensor[0,index] = 1.0

    return tensor

getCategoryIndex('English',all_categories)

encodeCategory('English',all_categories, len(all_categories))

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [221]:
# labels all_categories 
# dict category_lines

# LongTensor of second letter to end (EOS) for target
def targetTensor(line):
    letter_indexes = [all_letters.find(line[li]) for li in range(1, len(line))]
    letter_indexes.append(n_letters - 1) # EOS
    return torch.LongTensor(letter_indexes)

# each tuple in dataset = (direction vec + encoded letter)
dataset = []

all_categories_size = len(all_categories)

for category in all_categories:
    # print(category)
    category_encoded = encodeCategory(category, all_categories, all_categories_size)
    for word in category_lines[category]:
        word_begin = word
        word_end = word
        word_begin_tensor = lineToTensor(word_begin)
        word_end_tensor = targetTensor(word_end)

        dataset.append((category_encoded, word_begin_tensor, word_end_tensor))




In [222]:
import random
random.shuffle(dataset)
len(dataset)

20074

In [224]:
dataset[0][2].unsqueeze(-1)

tensor([[ 7],
        [ 4],
        [11],
        [58]])

In [273]:
model = Model(n_letters, 128, n_letters, len(all_categories))
criterion = nn.NLLLoss()

In [278]:
n_epochs = 10
learning_rate = 0.01

for epoch in range(n_epochs):
    for data in dataset:
        direction, letter_input, letter_target = data
        
        target = letter_target.unsqueeze(-1)
        model.zero_grad()
        loss = torch.tensor(0.0)
        hidden = model.init_hidden()
        for letter_index in range(letter_input.size(0)):
            output, hidden = model(direction, letter_input[letter_index], hidden)
            # print(output, letter_target)
            loss += criterion(output, target[letter_index])
        
        
        loss.backward()
        for p in model.parameters():
            p.data.add_(p.grad.data, alpha=-learning_rate)
    print('epoch...... ', epoch, ' loss...... ', loss)


epoch......  0  loss......  tensor(20.6776, grad_fn=<AddBackward0>)
epoch......  1  loss......  tensor(16.5187, grad_fn=<AddBackward0>)
epoch......  2  loss......  tensor(15.4509, grad_fn=<AddBackward0>)
epoch......  3  loss......  tensor(13.8330, grad_fn=<AddBackward0>)
epoch......  4  loss......  tensor(18.1565, grad_fn=<AddBackward0>)
epoch......  5  loss......  tensor(17.7501, grad_fn=<AddBackward0>)
epoch......  6  loss......  tensor(16.8465, grad_fn=<AddBackward0>)


KeyboardInterrupt: 

In [197]:
direction, letter_input, letter_target = dataset[5]
letter_target.unsqueeze(-1)

model.zero_grad()
loss = torch.tensor(0.0)
hidden = model.init_hidden()
output, hidden = model(direction, letter_input[letter_index], hidden)

output[0], letter_target[0]

MODEL out dropout tensor([[ 0.0574,  0.0211,  0.0228, -0.0312, -0.0062, -0.1348,  0.0022,  0.0489,
         -0.0158, -0.0484, -0.0719,  0.0843, -0.0000,  0.1007, -0.1185,  0.0501,
         -0.0481, -0.1293, -0.0082, -0.0264, -0.0244,  0.0880,  0.0227, -0.0000,
         -0.1021,  0.0621,  0.0880, -0.0335,  0.0808,  0.0581, -0.0517, -0.0532,
         -0.0482, -0.0418, -0.0252,  0.0937,  0.0238, -0.0197,  0.0867, -0.0613,
         -0.0557, -0.0036, -0.0829, -0.0090,  0.0684, -0.0000,  0.0000, -0.0107,
         -0.0363,  0.0151, -0.0607, -0.0394,  0.0309,  0.0516,  0.0468,  0.1260,
         -0.0000, -0.0405,  0.0000]], grad_fn=<MulBackward0>)
MODEL out softmax tensor([[-4.0200, -4.0564, -4.0546, -4.1086, -4.0837, -4.2123, -4.0753, -4.0285,
         -4.0932, -4.1259, -4.1493, -3.9931, -4.0774, -3.9767, -4.1959, -4.0274,
         -4.1255, -4.2068, -4.0856, -4.1039, -4.1018, -3.9894, -4.0548, -4.0774,
         -4.1795, -4.0153, -3.9895, -4.1109, -3.9967, -4.0193, -4.1292, -4.1306,
         -4

(tensor([-4.0200, -4.0564, -4.0546, -4.1086, -4.0837, -4.2123, -4.0753, -4.0285,
         -4.0932, -4.1259, -4.1493, -3.9931, -4.0774, -3.9767, -4.1959, -4.0274,
         -4.1255, -4.2068, -4.0856, -4.1039, -4.1018, -3.9894, -4.0548, -4.0774,
         -4.1795, -4.0153, -3.9895, -4.1109, -3.9967, -4.0193, -4.1292, -4.1306,
         -4.1257, -4.1192, -4.1027, -3.9837, -4.0537, -4.0971, -3.9907, -4.1387,
         -4.1332, -4.0810, -4.1603, -4.0865, -4.0091, -4.0774, -4.0774, -4.0881,
         -4.1137, -4.0623, -4.1381, -4.1168, -4.0465, -4.0258, -4.0307, -3.9514,
         -4.0774, -4.1179, -4.0774], grad_fn=<SelectBackward0>),
 tensor([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0.]]))

In [193]:
t = [-0.0015, -0.0601,  0.0482,  0.0075, -0.0703, -0.1492,  0.0607, -0.0599,
          0.0812,  0.0020, -0.0571,  0.0812,  0.0000, -0.0000, -0.0506,  0.0800,
          0.0110, -0.0294, -0.0439,  0.0814, -0.0000,  0.0894,  0.1321, -0.0575,
          0.0336, -0.0034, -0.0292,  0.0374,  0.0005, -0.0031,  0.0133, -0.0396,
         -0.0092,  0.0274,  0.0000,  0.1297, -0.1145,  0.0506,  0.0823,  0.0739,
         -0.0209, -0.0000,  0.0128, -0.0225, -0.0182,  0.0971, -0.0619, -0.0014,
          0.0000, -0.0671,  0.0667,  0.1022, -0.0792,  0.0738, -0.0000,  0.0098,
          0.0323,  0.0302, -0.0104]
t = torch.tensor(t)

a = F.log_softmax(t.float(), dim=0)
print(a)
print(a.argmax())
print(a[a.argmax()])

tensor([-4.0890, -4.1476, -4.0393, -4.0800, -4.1578, -4.2367, -4.0268, -4.1474,
        -4.0063, -4.0855, -4.1446, -4.0063, -4.0875, -4.0875, -4.1381, -4.0075,
        -4.0765, -4.1169, -4.1314, -4.0061, -4.0875, -3.9981, -3.9554, -4.1450,
        -4.0539, -4.0909, -4.1167, -4.0501, -4.0870, -4.0906, -4.0742, -4.1271,
        -4.0967, -4.0601, -4.0875, -3.9578, -4.2020, -4.0369, -4.0052, -4.0136,
        -4.1084, -4.0875, -4.0747, -4.1100, -4.1057, -3.9904, -4.1494, -4.0889,
        -4.0875, -4.1546, -4.0208, -3.9853, -4.1667, -4.0137, -4.0875, -4.0777,
        -4.0552, -4.0573, -4.0979])
tensor(22)
tensor(-3.9554)


In [289]:
# Just return an output given a line
def evaluate(direction, first_letter):
    with torch.no_grad():
        direction_tensor = encodeCategory(direction,all_categories,all_categories_size)
        letter_tensor = letterToTensor(first_letter)
        hidden = model.init_hidden()

        limit = 10

        outputs = ""
        for i in range(limit):
            output, hidden = model(direction_tensor, letter_tensor, hidden)
            position = torch.argmax(output)
            if position == len(all_letters):
                break
            outputs += all_letters[position]
            letter_tensor = output

        return outputs

In [312]:
evaluate('Japanese','O')

'kk'